In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import StructuredTool

llm = ChatOpenAI(temperature=0.1,)

def plus(a, b):
    return float(a) + float(b)

agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent = AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=[
        StructuredTool.from_function(
            func=plus,
            name="Sum Calculator",
            description="Use this to perform sums of two numbers.\
                This tool take two arguments, both should be numbers."
        ),])


prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"
agent.invoke(prompt)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import StructuredTool, Tool

llm = ChatOpenAI(temperature=0.1)

def plus(inputs):
    a, b = inputs.split(",")
    return float(a) + float(b)
    
#ZERO SHOT REACT DESCRIPTION = function의 argument를 1개만 받을 수 있음

agent = initialize_agent(
    llm = llm,
    verbose=True,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    tools=[Tool.from_function(
        func=plus,
        name="Sum Calculator",
        description="Use this to perform sums of two numbers. \
            Use this tool by sending a pair of number separated by comma. \
                \n Example: 1,2")])

prompt = "Cost is $400 + $500 + $300 + $250 + $100 + $150"

agent.invoke(prompt)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import StructuredTool, Tool, BaseTool
from pydantic import BaseModel, Field
from typing import Type


llm = ChatOpenAI(temperature=0.1)

def plus(a, b):
    return a + b

class CalculatorToolArgsSchema(BaseModel):
    a: float = Field(description="The First number")
    b: float = Field(description="The second number")

class CalculatorTool(BaseTool):
    
    name = "CalculatorTool"
    description="Use this to perform sums of two numbers. \
                The first and second arguments should be numbers. \
                Only receicves two arguments"
    args_schema: Type[CalculatorToolArgsSchema] = CalculatorToolArgsSchema

    def _run(self, a, b):
        return plus(a, b)
    
agent = initialize_agent(
    llm = llm,
    verbose = True,
    agent = AgentType.OPENAI_FUNCTIONS,
    handle_parshing_errors=True,
    tools = [CalculatorTool()]
)

prompt = "Cost is $400 + $500 + $300 + $250 + $100 + $150"

agent.invoke(prompt)

In [2]:
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper, GoogleSearchAPIWrapper
import os
import requests
from langchain_openai import ChatOpenAI



llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo-1106")
#Use Alpha Vantage API
alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")

class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for. Example query:\
            Stock Market Symbol for Google")

class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    """
    
    args_schema: Type[StockMarketSymbolSearchToolArgsSchema
                      ] = StockMarketSymbolSearchToolArgsSchema
    
    def _run(self, query):
        search = DuckDuckGoSearchAPIWrapper(max_results=1)
        return search.run(query)

class CompanyOverviewArgsSchema(BaseModel):
    symbol : str = Field(description="Stock Symbol of the company. Exmaple:\
        AAPL, TSLA")

class CompanyOverviewTool(BaseTool):
    name = "CompanyOverview"
    description = """
    Use this to get an overview of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}"
        r = requests.get(url)
        return r.json()

class CompanyIncomeStatementTool(BaseTool):
    name = "CompanyIncomeStatement"
    description = """
    Use this to get the income statement of a company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema
    
    def _run(self, symbol):
        url = f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}"
        r = requests.get(url)
        return r.json()["annualReports"]

class CompanyStockPerformanceTool(BaseTool):
    name = "CompanyStockPerformance"
    description = """
    Use this to get the weekly performance of a company stock.
    You should enter a stock symbol.
    """
    
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema
    
    def _run(self, symbol):
        url = f"https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={symbol}&apikey={alpha_vantage_api_key}"
        r = requests.get(url)
        return r.json()
    
agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
        CompanyOverviewTool(),
        CompanyIncomeStatementTool(),
        CompanyIncomeStatementTool(),
        
    ])

prompt = "Give me financial information on Tesla's stock, \
    considering its financials, income statements and \
    stock performance help me analyze if it's a potential good investment."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Tesla'}`


It's nonsensical because it's nonfactual—the reality is Tesla's customer retention is among the best and highest in the industry. Today, we are breaking ground on Tesla's in-house lithium refinery, located in the greater Corpus Christi area of Texas. Once complete, the facility will represent an investment of >$1B in Southwest Texas.
Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Tesla stock market symbol'}`
responded: It seems that there was an issue with retrieving the stock market symbol for Tesla. Let me try again to get the correct stock market symbol for Tesla.

The latest Tesla stock prices, stock quotes, news, and TSLA history to help you invest and trade smarter. ... Tesla's market capitalization is $562.14 B by 3.18 B shares outstanding.
Invoking: `CompanyOverview` with `{'symbol': 'TSLA'}`


{'Symbol': 'TSLA', 'AssetType': 'Common Stock', 'Name': 'Tesla Inc'

{'input': "Give me financial information on Tesla's stock,     considering its financials, income statements and     stock performance help me analyze if it's a potential good investment.",
 'output': "Here is the financial information for Tesla Inc. (TSLA):\n\n- Market Capitalization: $568.96 billion\n- EBITDA: $13.56 billion\n- P/E Ratio: 41.45\n- Revenue: $96.77 billion\n- Gross Profit: $17.66 billion\n- Operating Income: $8.89 billion\n- Net Income: $14.97 billion\n- Earnings Per Share (EPS): $4.31\n- Revenue Growth (YoY): 3.5%\n- Quarterly Earnings Growth (YoY): 111.5%\n- Profit Margin: 15.5%\n- Operating Margin: 8.2%\n- Return on Assets (ROA): 5.88%\n- Return on Equity (ROE): 27.4%\n- 52-Week High: $299.29\n- 52-Week Low: $152.37\n- 50-Day Moving Average: $208.30\n- 200-Day Moving Average: $234.93\n\nBased on this information, Tesla has shown strong revenue growth, positive net income, and a healthy profit margin. The company's stock performance has seen fluctuations over the pas